# Final Project

In [2]:
import serial, time

## Data Collection
Run below cells to collect twitch data

In [3]:
resolution = 100
duration = 10

# Setup Serial Communication
ser = serial.Serial('COM4') # TODO: Check if correct COM/TTYL port
ser.baudrate = 9600
start_time = time.time()

# Start Data Logging
f = open('{0}.txt'.format(start_time), "w")
while True:
    curr_time = time.time()
    if (curr_time - start_time > duration):
        break
    ser.write(str.encode('l'))
    f.write(str(ser.readline()))
ser.close()
f.close()


## Model Fitting
https://www.tensorflow.org/lite/microcontrollers 

https://github.com/tensorflow/decision-forests

In [6]:
import pandas as pd

#### Random Forest

In [11]:
# Load data
motion_data = pd.read_csv("./physionet.org/files/sleep-accel/1.0.0/motion/46343_acceleration.txt", delimiter=' ')
sleep_data = pd.read_csv("./physionet.org/files/sleep-accel/1.0.0/labels/46343_labeled_sleep.txt", delimiter=' ')

In [ ]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf

In [3]:
# Train model
model = tfdf.keras.RandomForestModel()
model.fit(train_data)

Use /tmp/tmpakfpji1_ as temporary training directory


2022-11-09 18:16:58.587796: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-09 18:16:58.587884: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-09 18:16:58.588024: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ed23fcd6f4be): /proc/driver/nvidia/version does not exist
2022-11-09 18:16:58.588328: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


NameError: name 'train_data' is not defined

In [7]:
model.summary()

NameError: name 'model' is not defined

In [9]:
model.evaluate(valid_data)

NameError: name 'model' is not defined